# Day 11: Space Police
On the way to Jupiter, you're pulled over by the Space Police.

"Attention, unmarked spacecraft! You are in violation of Space Law! All spacecraft must have a clearly visible registration identifier! You have 24 hours to comply or be sent to Space Jail!"

Not wanting to be sent to Space Jail, you radio back to the Elves on Earth for help. Although it takes almost three hours for their reply signal to reach you, they send instructions for how to power up the emergency hull painting robot and even provide a small Intcode program (your puzzle input) that will cause it to paint your ship appropriately.

There's just one problem: you don't have an emergency hull painting robot.

You'll need to build a new emergency hull painting robot. The robot needs to be able to move around on the grid of square panels on the side of your ship, detect the color of its current panel, and paint its current panel black or white. (All of the panels are currently black.)

The Intcode program will serve as the brain of the robot. The program uses input instructions to access the robot's camera: provide 0 if the robot is over a black panel or 1 if the robot is over a white panel. Then, the program will output two values:

First, it will output a value indicating the color to paint the panel the robot is over: 0 means to paint the panel black, and 1 means to paint the panel white.
Second, it will output a value indicating the direction the robot should turn: 0 means it should turn left 90 degrees, and 1 means it should turn right 90 degrees.
After the robot turns, it should always move forward exactly one panel. The robot starts facing up.

The robot will continue running for a while like this and halt when it is finished drawing. Do not restart the Intcode computer inside the robot during this process.

For example, suppose the robot is about to start running. Drawing black panels as ., white panels as #, and the robot pointing the direction it is facing (< ^ > v), the initial state and region near the robot looks like this:

```
.....
.....
..^..
.....
.....
```

The panel under the robot (not visible here because a ^ is shown instead) is also black, and so any input instructions at this point should be provided 0. Suppose the robot eventually outputs 1 (paint white) and then 0 (turn left). After taking these actions and moving forward one panel, the region now looks like this:

```
.....
.....
.<#..
.....
.....
```

Input instructions should still be provided 0. Next, the robot might output 0 (paint black) and then 0 (turn left):

```
.....
.....
..#..
.v...
.....
```

After more outputs (1,0, 1,0):

```
.....
.....
..^..
.##..
.....
```

The robot is now back where it started, but because it is now on a white panel, input instructions should be provided 1. After several more outputs (0,1, 1,0, 1,0), the area looks like this:

```
.....
..<#.
...#.
.##..
.....
```

Before you deploy the robot, you should probably have an estimate of the area it will cover: specifically, you need to know the number of panels it paints at least once, regardless of color. In the example above, the robot painted 6 panels at least once. (It painted its starting panel twice, but that panel is still only counted once; it also never painted the panel it ended on.)

Build a new emergency hull painting robot and run the Intcode program on it. How many panels does it paint at least once?

In [1]:
from intcode import *

In [19]:
import enum
import collections

In [40]:
class Orientation(enum.Enum):
    NORTH = 0
    EAST = 1
    SOUTH = 2
    WEST = 3
    
    def turn_right(self):
        return Orientation((self.value + 1) % 4)
    
    def turn_left(self):
        return Orientation((self.value + 3) % 4)
    
    @property
    def unit(self):
        return [
            (0, -1),
            (1, 0),
            (0, 1),
            (-1, 0),
        ][self.value]

In [14]:
def add_positions(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return (x1 + x2, y1 + y2)

In [24]:
class PaintColor(enum.Enum):
    BLACK = 0
    WHITE = 1
    
class OutputState(enum.Enum):
    PAINT_COLOR = enum.auto()
    DIRECTION = enum.auto()

In [28]:
class PaintTheShip:
    def reset(self):
        self.orientation = Orientation.NORTH
        self.position = (0, 0)
        self.canvas = collections.defaultdict(lambda: PaintColor.BLACK)
        self.output_state = OutputState.PAINT_COLOR
        
    def input(self):
        return self.canvas[self.position].value
    
    def output(self, value):
        if self.output_state is OutputState.PAINT_COLOR:
            self.canvas[self.position] = PaintColor(value)
            self.output_state = OutputState.DIRECTION
        else:
            self.orientation = self.orientation.turn_right() if value else self.orientation.turn_left()
            self.position = add_positions(self.position, self.orientation.unit)
            self.output_state = OutputState.PAINT_COLOR

In [42]:
env = PaintTheShip()

In [7]:
import tools

In [8]:
data = tools.get_data(11)

In [9]:
interpreter = IntcodeInterpreter()

In [43]:
interpreter.evaluate(make_tape(data), env, debug=True)

CallContext: _input [8](->0)
CallContext: jump_if_true [8](->0) 291L
CallContext: jump_if_false 0L 11L
CallContext: output 1L
CallContext: output 0L
CallContext: _input [8](->0)
CallContext: multiply [8](->0) -1L [10](->0)
CallContext: add 1L [10](->0) [10](->0)
CallContext: output [10](->1)
CallContext: equals 0L [8](->0) [10](->1)
CallContext: output [10](->1)
CallContext: multiply [8](->0) 1L [28](->0)
CallContext: add [1003](->0) [20](->10) [10](->1)
CallContext: multiply [1103](->0) [19](->-1) [10](->10)
CallContext: _input [8](->0)
CallContext: multiply [8](->0) -1L [10](->0)
CallContext: add [10](->0) 1L [10](->0)
CallContext: output [10](->1)
CallContext: equals [8](->0) 0L [10](->1)
CallContext: output [10](->1)
CallContext: add [8](->0) 0L [59](->0)
CallContext: add [1004](->0) [3](->8) [10](->1)
CallContext: _input [8](->0)
CallContext: multiply -1L [8](->0) [10](->8)
CallContext: add [10](->0) 1L [10](->0)
CallContext: output [10](->1)
CallContext: equals 0L [8](->0) [10](-

CallContext: add 0L [8](->1) [114](->1)
CallContext: _input [8](->1)
CallContext: multiply [8](->0) -1L [10](->1)
CallContext: add 1L [10](->0) [10](->0)
CallContext: output [10](->1)
CallContext: equals 1L [8](->0) [10](->1)
CallContext: output [10](->0)
CallContext: add 0L [8](->0) [135](->1)
CallContext: _input [8](->0)
CallContext: multiply [8](->1) -1L [10](->0)
CallContext: add [10](->-1) 1L [10](->-1)
CallContext: output [10](->0)
CallContext: equals [8](->1) 0L [10](->0)
CallContext: output [10](->0)
CallContext: multiply 1L [8](->1) [158](->0)
CallContext: multiply [9](->29) [9](->29) [10](->0)
CallContext: multiply [2](->1005) [10](->841) [10](->841)
CallContext: _input [8](->1)
CallContext: multiply [8](->1) -1L [10](->845205)
CallContext: add [10](->-1) 1L [10](->-1)
CallContext: output [10](->0)
CallContext: equals [8](->1) 0L [10](->0)
CallContext: output [10](->0)
CallContext: add 0L [8](->1) [188](->0)
CallContext: jump_if_false [0](->3) 56L
CallContext: _input [8](->1)

CallContext: output [10](->1)
CallContext: equals [8](->0) 0L [10](->1)
CallContext: output [10](->1)
CallContext: multiply 1L [8](->0) [158](->0)
CallContext: multiply [9](->58) [9](->58) [10](->1)
CallContext: multiply [2](->1005) [10](->3364) [10](->3364)
CallContext: _input [8](->0)
CallContext: multiply [8](->0) -1L [10](->3380820)
CallContext: add [10](->0) 1L [10](->0)
CallContext: output [10](->1)
CallContext: equals [8](->0) 1L [10](->1)
CallContext: output [10](->0)
CallContext: add 0L [8](->0) [188](->1)
CallContext: jump_if_false [0](->3) 56L
CallContext: _input [8](->0)
CallContext: multiply [8](->0) -1L [10](->0)
CallContext: add [10](->0) 1L [10](->0)
CallContext: output [10](->1)
CallContext: equals 1L [8](->0) [10](->1)
CallContext: output [10](->0)
CallContext: add [8](->0) 0L [212](->1)
CallContext: jump_if_false [0](->3) 76L
CallContext: multiply [1005](->0) [8](->0) [10](->0)
CallContext: _input [8](->0)
CallContext: multiply -1L [8](->0) [10](->0)
CallContext: add

CallContext: jump_if_false [0](->3) 56L
CallContext: _input [8](->1)
CallContext: multiply [8](->1) -1L [10](->0)
CallContext: add [10](->-1) 1L [10](->-1)
CallContext: output [10](->0)
CallContext: equals 1L [8](->1) [10](->0)
CallContext: output [10](->1)
CallContext: add [8](->1) 0L [212](->1)
CallContext: jump_if_false [0](->3) 76L
CallContext: multiply [1005](->0) [8](->1) [10](->1)
CallContext: _input [8](->1)
CallContext: multiply -1L [8](->1) [10](->0)
CallContext: add [10](->-1) 1L [10](->-1)
CallContext: output [10](->0)
CallContext: equals 1L [8](->1) [10](->0)
CallContext: output [10](->1)
CallContext: add [8](->1) 0L [241](->1)
CallContext: _input [8](->1)
CallContext: multiply -1L [8](->0) [10](->1)
CallContext: add 1L [10](->0) [10](->0)
CallContext: output [10](->1)
CallContext: equals [8](->0) 0L [10](->1)
CallContext: output [10](->1)
CallContext: multiply [8](->0) 1L [264](->0)
CallContext: jump_if_false [0](->3) 95L
CallContext: add [1001](->0) [12](->1) [10](->1)
C

CallContext: add [8](->0) 0L [241](->1)
CallContext: _input [8](->0)
CallContext: multiply -1L [8](->1) [10](->0)
CallContext: add 1L [10](->-1) [10](->-1)
CallContext: output [10](->0)
CallContext: equals [8](->1) 1L [10](->0)
CallContext: output [10](->1)
CallContext: multiply [8](->1) 1L [264](->1)
CallContext: jump_if_false [0](->3) 95L
CallContext: add [1001](->0) [12](->1) [10](->1)
CallContext: add 1L [9](->115) [9](->115)
CallContext: less_than [9](->116) 933L [10](->1)
CallContext: jump_if_true [10](->1) 15L
CallContext: _input [8](->1)
CallContext: multiply [8](->0) -1L [10](->1)
CallContext: add 1L [10](->0) [10](->0)
CallContext: output [10](->1)
CallContext: equals 1L [8](->0) [10](->1)
CallContext: output [10](->0)
CallContext: multiply [8](->0) 1L [28](->1)
CallContext: add [1003](->0) [20](->10) [10](->0)
CallContext: multiply [1103](->0) [19](->-1) [10](->10)
CallContext: _input [8](->0)
CallContext: multiply [8](->1) -1L [10](->0)
CallContext: add [10](->-1) 1L [10](-

...

CallContext: equals [8](->0) 0L [10](->1)
CallContext: output [10](->1)
CallContext: multiply 1L [8](->0) [158](->0)
CallContext: multiply [9](->834) [9](->834) [10](->1)
CallContext: multiply [2](->1005) [10](->695556) [10](->695556)
CallContext: _input [8](->0)
CallContext: multiply [8](->1) -1L [10](->699033780)
CallContext: add [10](->-1) 1L [10](->-1)
CallContext: output [10](->0)
CallContext: equals [8](->1) 1L [10](->0)
CallContext: output [10](->1)
CallContext: add 0L [8](->1) [188](->1)
CallContext: jump_if_false [0](->3) 56L
CallContext: _input [8](->1)
CallContext: multiply [8](->1) -1L [10](->1)
CallContext: add [10](->-1) 1L [10](->-1)
CallContext: output [10](->0)
CallContext: equals 0L [8](->1) [10](->0)
CallContext: output [10](->0)
CallContext: add [8](->1) 0L [212](->0)
CallContext: jump_if_false [0](->3) 76L
CallContext: multiply [1005](->0) [8](->1) [10](->0)
CallContext: _input [8](->1)
CallContext: multiply -1L [8](->1) [10](->0)
CallContext: add [10](->-1) 1L [10

CallContext: _input [8](->0)
CallContext: multiply [8](->1) -1L [10](->0)
CallContext: add [10](->-1) 1L [10](->-1)
CallContext: output [10](->0)
CallContext: equals [8](->1) 1L [10](->0)
CallContext: output [10](->1)
CallContext: add [8](->1) 0L [59](->1)
CallContext: add [1004](->0) [3](->8) [10](->1)
CallContext: _input [8](->1)
CallContext: multiply -1L [8](->0) [10](->8)
CallContext: add [10](->0) 1L [10](->0)
CallContext: output [10](->1)
CallContext: equals 0L [8](->0) [10](->1)
CallContext: output [10](->1)
CallContext: add [8](->0) 0L [84](->0)
CallContext: jump_if_false [0](->3) 3L
CallContext: add [1102](->0) [12](->1) [10](->1)
CallContext: _input [8](->0)
CallContext: multiply [8](->1) -1L [10](->1)
CallContext: add 1L [10](->-1) [10](->-1)
CallContext: output [10](->0)
CallContext: equals [8](->1) 0L [10](->0)
CallContext: output [10](->0)
CallContext: add 0L [8](->1) [114](->0)
CallContext: _input [8](->1)
CallContext: multiply [8](->1) -1L [10](->0)
CallContext: add 1L 

CallContext: add [8](->1) 0L [59](->0)
CallContext: add [1004](->0) [3](->8) [10](->0)
CallContext: _input [8](->1)
CallContext: multiply -1L [8](->0) [10](->8)
CallContext: add [10](->0) 1L [10](->0)
CallContext: output [10](->1)
CallContext: equals 0L [8](->0) [10](->1)
CallContext: output [10](->1)
CallContext: add [8](->0) 0L [84](->0)
CallContext: jump_if_false [0](->3) 3L
CallContext: add [1102](->0) [12](->1) [10](->1)
CallContext: _input [8](->0)
CallContext: multiply [8](->1) -1L [10](->1)
CallContext: add 1L [10](->-1) [10](->-1)
CallContext: output [10](->0)
CallContext: equals [8](->1) 0L [10](->0)
CallContext: output [10](->0)
CallContext: add 0L [8](->1) [114](->0)
CallContext: _input [8](->1)
CallContext: multiply [8](->1) -1L [10](->0)
CallContext: add 1L [10](->-1) [10](->-1)
CallContext: output [10](->0)
CallContext: equals 0L [8](->1) [10](->0)
CallContext: output [10](->0)
CallContext: add 0L [8](->1) [135](->0)
CallContext: _input [8](->1)
CallContext: multiply [8]

CallContext: add [10](->-1) 1L [10](->-1)
CallContext: output [10](->0)
CallContext: equals [8](->1) 1L [10](->0)
CallContext: output [10](->1)
CallContext: multiply 1L [8](->1) [158](->1)
CallContext: multiply [9](->919) [9](->919) [10](->1)
CallContext: multiply [2](->1005) [10](->844561) [10](->844561)
CallContext: _input [8](->1)
CallContext: multiply [8](->0) -1L [10](->848783805)
CallContext: add [10](->0) 1L [10](->0)
CallContext: output [10](->1)
CallContext: equals [8](->0) 1L [10](->1)
CallContext: output [10](->0)
CallContext: add 0L [8](->0) [188](->1)
CallContext: jump_if_false [0](->3) 56L
CallContext: _input [8](->0)
CallContext: multiply [8](->1) -1L [10](->0)
CallContext: add [10](->-1) 1L [10](->-1)
CallContext: output [10](->0)
CallContext: equals 0L [8](->1) [10](->0)
CallContext: output [10](->0)
CallContext: add [8](->1) 0L [212](->0)
CallContext: jump_if_false [0](->3) 76L
CallContext: multiply [1005](->0) [8](->1) [10](->0)
CallContext: _input [8](->1)
CallConte


            InterpreterContext(
                index=290,
                state=99,
                relative_base=0,
                tape=Tape[3, 8, 1005, 8, 291, 1106, 0, 11, 1, 933, 0, 104, 1, 104, 0, 3, 8, 1002, 8, -1, 10, 101, 1, 10, 10, 4, 10, 108, 0, 8, 10, 4, 10, 1002, 8, 1, 28, 1, 1003, 20, 10, 2, 1103, 19, 10, 3, 8, 1002, 8, -1, 10, 1001, 10, 1, 10, 4, 10, 1008, 8, 1, 10, 4, 10, 1001, 8, 0, 59, 1, 1004, 3, 10, 3, 8, 102, -1, 8, 10, 1001, 10, 1, 10, 4, 10, 108, 1, 8, 10, 4, 10, 1001, 8, 0, 84, 1006, 0, 3, 1, 1102, 12, 10, 3, 8, 1002, 8, -1, 10, 101, 1, 10, 10, 4, 10, 1008, 8, 1, 10, 4, 10, 101, 0, 8, 114, 3, 8, 1002, 8, -1, 10, 101, 1, 10, 10, 4, 10, 108, 0, 8, 10, 4, 10, 101, 0, 8, 135, 3, 8, 1002, 8, -1, 10, 1001, 10, 1, 10, 4, 10, 1008, 8, 1, 10, 4, 10, 102, 1, 8, 158, 2, 9, 9, 10, 2, 2, 10, 10, 3, 8, 1002, 8, -1, 10, 1001, 10, 1, 10, 4, 10, 1008, 8, 1, 10, 4, 10, 101, 0, 8, 188, 1006, 0, 56, 3, 8, 1002, 8, -1, 10, 1001, 10, 1, 10, 4, 10, 108, 0, 8, 10, 4, 10, 1001, 8, 0, 

In [46]:
len(env.canvas)

2319

### Part Two
You're not sure what it's trying to paint, but it's definitely not a registration identifier. The Space Police are getting impatient.

Checking your external ship cameras again, you notice a white panel marked "emergency hull painting robot starting panel". The rest of the panels are still black, but it looks like the robot was expecting to start on a white panel, not a black one.

Based on the Space Law Space Brochure that the Space Police attached to one of your windows, a valid registration identifier is always eight capital letters. After starting the robot on a single white panel instead, what registration identifier does it paint on your hull?

In [47]:
class PaintTheShip:
    def __init__(self, starting_canvas):
        self._starting_canvas = dict(starting_canvas)
        
    def reset(self):
        self.orientation = Orientation.NORTH
        self.position = (0, 0)
        self.canvas = collections.defaultdict(lambda: PaintColor.BLACK)
        self.canvas.update(self._starting_canvas)
        self.output_state = OutputState.PAINT_COLOR
        
    def input(self):
        return self.canvas[self.position].value
    
    def output(self, value):
        if self.output_state is OutputState.PAINT_COLOR:
            self.canvas[self.position] = PaintColor(value)
            self.output_state = OutputState.DIRECTION
        else:
            self.orientation = self.orientation.turn_right() if value else self.orientation.turn_left()
            self.position = add_positions(self.position, self.orientation.unit)
            self.output_state = OutputState.PAINT_COLOR

In [48]:
env = PaintTheShip({(0, 0): PaintColor.WHITE})

In [49]:
interpreter.evaluate(make_tape(data), env)


            InterpreterContext(
                index=411,
                state=99,
                relative_base=613,
                tape=Tape[3, 8, 1005, 8, 291, 1106, 0, 11, 1, 0, 1, 104, 1, 104, 0, 3, 8, 1002, 8, -1, 10, 101, 1, 10, 10, 4, 10, 108, 0, 8, 10, 4, 10, 1002, 8, 1, 28, 1, 1003, 20, 10, 2, 1103, 19, 10, 3, 8, 1002, 8, -1, 10, 1001, 10, 1, 10, 4, 10, 1008, 8, 0, 10, 4, 10, 1001, 8, 0, 59, 1, 1004, 3, 10, 3, 8, 102, -1, 8, 10, 1001, 10, 1, 10, 4, 10, 108, 0, 8, 10, 4, 10, 1001, 8, 0, 84, 1006, 0, 3, 1, 1102, 12, 10, 3, 8, 1002, 8, -1, 10, 101, 1, 10, 10, 4, 10, 1008, 8, 1, 10, 4, 10, 101, 0, 8, 114, 3, 8, 1002, 8, -1, 10, 101, 1, 10, 10, 4, 10, 108, 1, 8, 10, 4, 10, 101, 0, 8, 135, 3, 8, 1002, 8, -1, 10, 1001, 10, 1, 10, 4, 10, 1008, 8, 0, 10, 4, 10, 102, 1, 8, 158, 2, 9, 9, 10, 2, 2, 10, 10, 3, 8, 1002, 8, -1, 10, 1001, 10, 1, 10, 4, 10, 1008, 8, 1, 10, 4, 10, 101, 0, 8, 188, 1006, 0, 56, 3, 8, 1002, 8, -1, 10, 1001, 10, 1, 10, 4, 10, 108, 1, 8, 10, 4, 10, 1001, 8, 0, 

In [50]:
env.canvas

defaultdict(<function __main__.PaintTheShip.reset.<locals>.<lambda>()>,
            {(0, 0): <PaintColor.BLACK: 0>,
             (1, 0): <PaintColor.WHITE: 1>,
             (1, 1): <PaintColor.WHITE: 1>,
             (2, 1): <PaintColor.BLACK: 0>,
             (2, 0): <PaintColor.BLACK: 0>,
             (3, 0): <PaintColor.BLACK: 0>,
             (3, 1): <PaintColor.BLACK: 0>,
             (4, 1): <PaintColor.WHITE: 1>,
             (4, 0): <PaintColor.WHITE: 1>,
             (5, 0): <PaintColor.BLACK: 0>,
             (5, 1): <PaintColor.BLACK: 0>,
             (6, 1): <PaintColor.WHITE: 1>,
             (6, 0): <PaintColor.WHITE: 1>,
             (7, 0): <PaintColor.WHITE: 1>,
             (7, 1): <PaintColor.BLACK: 0>,
             (8, 1): <PaintColor.BLACK: 0>,
             (8, 0): <PaintColor.WHITE: 1>,
             (9, 0): <PaintColor.WHITE: 1>,
             (9, 1): <PaintColor.BLACK: 0>,
             (10, 1): <PaintColor.BLACK: 0>,
             (10, 0): <PaintColor.BLACK: 0>,
  

In [54]:
def print_canvas(canvas):
    min_x = min(point[0] for point in canvas)
    max_x = max(point[0] for point in canvas)
    min_y = min(point[1] for point in canvas)
    max_y = max(point[1] for point in canvas)
    
    for j in range(min_y, max_y + 1):
        row = ''
        for i in range(min_x, max_x + 1):
            row += '.' if canvas.get((i, j), PaintColor.BLACK) is PaintColor.BLACK else '#'
        print(row)

In [55]:
print_canvas(env.canvas)

.#..#.####.###..###..###..####..##....##...
.#..#.#....#..#.#..#.#..#.#....#..#....#...
.#..#.###..#..#.#..#.#..#.###..#.......#...
.#..#.#....###..###..###..#....#.##....#...
.#..#.#....#.#..#....#.#..#....#..#.#..#...
..##..####.#..#.#....#..#.#.....###..##....
